<a href="https://www.kaggle.com/code/zoyaansari/nlp-ml?scriptVersionId=118707199" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing,

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/pickle-preprocesssed-files/train.pkl
/kaggle/input/pickle-preprocesssed-files/test.pkl


In [2]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
!pip install autocorrect
from autocorrect import Speller
import string
import timeit
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=23fd797992476c7a355a3632d8336b29ccff29215c8ef959e99b3fda6dccd7ee
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect


In [4]:
def remove_newlines_tabs(text):
    """
    This function will remove all the occurrences of newlines, tabs, and combinations like: \\n, \\.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of newlines, tabs, \\n, \\ characters.
        
    Example:
    Input : This is her \\ first day at this place.\n Please,\t Be nice to her.\\n
    Output : This is her first day at this place. Please, Be nice to her. 
    
    """
    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return Formatted_text

remove_newlines_tabs("This is her \\ first day at this place.\n Please,\t Be nice to her.\\n")

'This is her   first day at this place.  Please,  Be nice to her. '

In [5]:
def strip_html_tags(text):
    """ 
    This function will remove all the occurrences of html tags from the text.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of html tags.
        
    Example:
    Input : This is a nice place to live. 
    Output : This is a nice place to live.  
    """
    # Initiating BeautifulSoup object soup.
    soup = BeautifulSoup(text, "html.parser")
    # Get all the text other than html tags.
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


strip_html_tags(" This is a <b>nice</b> place to live. ")

' This is a  nice  place to live. '

In [6]:
import re
def remove_links(text):
    """
    This function will remove all the occurrences of links.
    
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after removal of all types of links.
        
    Example:
    Input : To know more about cats and food & website: catster.com  visit: https://catster.com//how-to-feed-cats
    Output : To know more about cats and food & website: visit:     
    
    """

    # Removing all the occurrences of links that starts with https
    remove_https = re.sub(r'http\S+', '', text)
    # Remove all the occurrences of text that ends with .com
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com

remove_links(" website: catster.com  visit: https://catster.com//how-to-feed-cats")

' website:   visit: '

In [7]:

def remove_whitespace(text):
    """ This function will remove 
        extra whitespaces from the text
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" after extra whitespaces removed .
        
    Example:
    Input : How   are   you   doing   ?
    Output : How are you doing ?     
        
    """
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text    

remove_whitespace("How   are   you   doing? (pakistan)ABC")

'How are you doing ?  (pakistan) ABC'

In [8]:
# Code for accented characters removal
def accented_characters_removal(text):
    # this is a docstring
    """
    The function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.
        
    Example:
    Input : Málaga, àéêöhello
    Output : Malaga, aeeohello    
        
    """
    # Remove accented characters from text using unidecode.
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text)
    return text

accented_characters_removal("Málaga, àéêöhello")

'Malaga, aeeohello'

In [9]:
# Code for text lowercasing
def lower_casing_text(text):
    
    """
    The function will convert text into lower case.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
         value: text in lowercase
         
    Example:
    Input : The World is Full of Surprises!
    Output : the world is full of surprises!
    
    """
    # Convert text to lower case
    # lower() - It converts all upperase letter of given string to lowercase.
    text = text.lower()
    return text

lower_casing_text("Pakistan Zinda BAD!")

'pakistan zinda bad!'

In [10]:
def reducing_incorrect_character_repeatation(text):
    """
    This Function will reduce repeatition to two characters 
    for alphabets and to one character for punctuations.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Finally formatted text with alphabets repeating to 
        two characters & punctuations limited to one repeatition 
        
    Example:
    Input : Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)
    Output : Reallyy, Greeaatt !?.;:)
    
    """
    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

reducing_incorrect_character_repeatation("Realllllllllyyyyy,        Greeeeaaaatttt   !!!!?....;;;;:)")

'Reallyy, Greeaatt !?.;:)'

Explanation for using some symbols in above regex expression
\1 —> is equivalent to re.search(...). group(1). It Refers to first capturing group. \1 matches the exact same text that was matched by the first capturing group.

{1,} --> It means we are matching for repeatation that occurs more than one times.

DOTALL -> It matches newline character as well unlike dot operator which matches everything in the given text except newline character.

sub() --> This function is used to replace occurrences of a particular sub-string with another sub-string. This function takes as input the following: The sub-string to replace. The sub-string to replace with.

r'\1\1' --> It limits all the repeatation to two characters.

r'\1' --> Limits all the repeatation to only one character.

{2,} --> It means to match for repeatation that occurs more than two times

In [11]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
# The code for expanding contraction words
def expand_contractions(text, contraction_mapping =  CONTRACTION_MAP):
    """expand shortened words to the actual form.
       e.g. don't to do not
    
       arguments:
            input_text: "text" of type "String".
         
       return:
            value: Text with expanded form of shorthened words.
        
       Example: 
       Input : ain't, aren't, can't, cause, can't've
       Output :  is not, are not, cannot, because, cannot have 
    
     """
    # Tokenizing text into tokens.
    list_Of_tokens = text.split(' ')

    # Checking for whether the given token matches with the Key & replacing word with key's value.
    
    # Check whether Word is in lidt_Of_tokens or not.
    for Word in list_Of_tokens: 
        # Check whether found word is in dictionary "Contraction Map" or not as a key. 
         if Word in CONTRACTION_MAP: 
                # If Word is present in both dictionary & list_Of_tokens, replace that word with the key value.
                list_Of_tokens = [item.replace(Word, CONTRACTION_MAP[Word]) for item in list_Of_tokens]
                
    # Converting list of tokens to String.
    String_Of_tokens = ' '.join(str(e) for e in list_Of_tokens) 
    return String_Of_tokens    

expand_contractions("ain't , aren't , can't , cause , can't've")
   

"is not , are not , cannot , cause , cannot've"

In [12]:
# The code for removing special characters
def removing_special_characters(text):
    """Removing all the special characters except the one that is passed within 
       the regex to match, as they have imp meaning in the text provided.
   
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text with removed special characters that don't require.
        
    Example: 
    Input : Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) 
    Output :  Hello, Kajal. This is $100.05 : the payment that you will recieve! Is this okay?
    
   """
    # The formatted text after removing not necessary punctuations.
    Formatted_Text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text) 
    # In the above regex expression,I am providing necessary set of punctuations that are frequent in this particular dataset.
    return Formatted_Text

removing_special_characters(" Hello, K-a-j-a-l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) ")

' Hello, K a j a l. Thi*s is $100.05 : the payment that you will recieve! (Is this okay?) '

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
# The code for removing stopwords
stoplist = stopwords.words('english') 

stoplist = set(stoplist)
def removing_stopwords(text):
    """This function will remove stopwords which doesn't add much meaning to a sentence 
       & they can be remove safely without comprimising meaning of the sentence.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after omitted all stopwords.
        
    Example: 
    Input : This is Kajal from delhi who came here to study.
    Output : ["'This", 'Kajal', 'delhi', 'came', 'study', '.', "'"] 
    
   """
    # repr() function actually gives the precise information about the string
    text = repr(text)
    # Text without stopwords
    No_StopWords = [word for word in word_tokenize(text) if word.lower() not in stoplist ]
    # Convert list of tokens_without_stopwords to String type.
    words_string = ' '.join(No_StopWords)    
    return words_string

removing_stopwords("This is Kajal from delhi who came here to study.")

"'This Kajal delhi came study . '"

In [15]:
# The code for spelling corrections
def spelling_correction(text):
    ''' 
    This function will correct spellings.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after corrected spellings.
        
    Example: 
    Input : This is Oberois from Dlhi who came heree to studdy.
    Output : This is Oberoi from Delhi who came here to study.
      
    
    '''
    # Check for spellings in English language
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text

spelling_correction("This is Oberois from Dlhi who came heree to studdy.")

'This is Oberoi from Delhi who came here to study.'

In [16]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [17]:
# The code for lemmatization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatization(text):
    """This function converts word to their root words 
       without explicitely cut down as done in stemming.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text having root words only, no tense form, no plural forms
        
    Example: 
    Input : text reduced 
    Output :  text reduce
    
   """
    # Converting words to their root forms
    lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
    return lemma

lemmatization("ext having root words only, no tense form, no plural forms")

['ext',
 'have',
 'root',
 'word',
 'only,',
 'no',
 'tense',
 'form,',
 'no',
 'plural',
 'form']

In [18]:
lemmatizer.lemmatize('educations','v')

'educations'

In [19]:
# Writing main function to merge all the preprocessing steps.
def text_preprocessing(text, accented_chars=True, contractions=True, lemmatization_allow = True,
                        extra_whitespace=True, newlines_tabs=True, repeatition=True, 
                       lowercase=True, punctuations=True, mis_spell=True,
                       remove_html=True, links=True,  special_chars=True,
                       stop_words=True):
    """
    This function will preprocess input text and return
    the clean text.
    """
    
    if newlines_tabs == True: #remove newlines & tabs.
        Data = remove_newlines_tabs(text)
        
    if remove_html == True: #remove html tags
        Data = strip_html_tags(Data)
        
    if links == True: #remove links
        Data = remove_links(Data)
        
    if extra_whitespace == True: #remove extra whitespaces
        Data = remove_whitespace(Data)
        
    if accented_chars == True: #remove accented characters
        Data = accented_characters_removal(Data)
        
    if lowercase == True: #convert all characters to lowercase
        Data = lower_casing_text(Data)
        
    if repeatition == True: #Reduce repeatitions   
        Data = reducing_incorrect_character_repeatation(Data)
        
    if contractions == True: #expand contractions
        Data = expand_contractions(Data)
    
    if punctuations == True: #remove punctuations
        Data = removing_special_characters(Data)
    
    stoplist = stopwords.words('english') 
    stoplist = set(stoplist)
    
    if stop_words == True: #Remove stopwords
        Data = removing_stopwords(Data)
        
    spell = Speller(lang='en')
    
    if mis_spell == True: #Check for mis-spelled words & correct them.
        Data = spelling_correction(Data)
        
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
#     print(Data)
    if lemmatization_allow == True: #Converts words to lemma form.
        Data = lemmatization(Data)
    
           
    return Data
text_preprocessing(' would have a hard time sitting through this one dogs ')

["'", 'would', 'hard', 'time', 'sit', 'one', 'dog', "'"]

In [20]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')


In [21]:
%%time
train['tokens']= train.text.apply(text_preprocessing)

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:439: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning


CPU times: user 37min 58s, sys: 1min 51s, total: 39min 49s
Wall time: 39min 51s


In [22]:
%%time
test['tokens']= test.text.apply(text_preprocessing)

CPU times: user 16min 48s, sys: 53.8 s, total: 17min 42s
Wall time: 17min 43s


In [23]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

train =pd.read_pickle('/kaggle/working/train.pkl')

test = pd.read_pickle('/kaggle/working/test.pkl')

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog."]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_,"Vocab")
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape,"Shape")
print(type(vector),"Type")
print(vector.toarray(),"Array")

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1} Vocab
(1, 8) Shape
<class 'scipy.sparse.csr.csr_matrix'> Type
[[1 1 1 1 1 1 1 2]] Array


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = ["The quick brown fox jumped over the lazy dog.",
 "The dog.",
 "The fox"]
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector = vectorizer.transform([text[0]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())#tfidf on  youtube

{'the': 7, 'quick': 6, 'brown': 0, 'fox': 2, 'jumped': 3, 'over': 5, 'lazy': 4, 'dog': 1}
[1.69314718 1.28768207 1.28768207 1.69314718 1.69314718 1.69314718
 1.69314718 1.        ]
(1, 8)
[[0.36388646 0.27674503 0.27674503 0.36388646 0.36388646 0.36388646
  0.36388646 0.42983441]]


In [26]:
train['tokens1'] = train.tokens.apply(lambda x:" ".join(x))
train.head()

,id,keyword,location,text,target,tokens,tokens1
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"['our, deeds, reason, earthquake, may, alla, f...",'our deeds reason earthquake may alla forgive ...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"['forest, fire, near, la, range, task, ., cana...",'forest fire near la range task . canada '
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[``, residents, ask, 'shelter, place, ', notif...",`` residents ask 'shelter place ' notify offic...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"['13,000, people, receive, wildfires, evacuati...","'13,000 people receive wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"['just, get, send, photo, ruby, alaska, smoke,...",'just get send photo ruby alaska smoke wildfir...


In [27]:
test['tokens1'] = test.tokens.apply(lambda x:" ".join(x))
test.head()

,id,keyword,location,text,tokens,tokens1
0,0,NaN,NaN,Just happened a terrible car crash,"['just, happen, terrible, car, crash, ']",'just happen terrible car crash '
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...","['heard, earthquake, different, cities, ,, sta...","'heard earthquake different cities , stay safe..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...","['there, forest, fire, spot, pond, ,, geese, f...","'there forest fire spot pond , geese flee acro..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,"['apocalypse, light, ., speak, wildfires, ']",'apocalypse light . speak wildfires '
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,"['typhoon, soudelor, kill, 28, china, taiwan, ']",'typhoon soudelor kill 28 china taiwan '


In [28]:
train = train[['text', 'tokens', 'tokens1', 'target']]

In [29]:
test = test[['text', 'tokens', 'tokens1']]

In [30]:
train.head()

,text,tokens,tokens1,target
0,Our Deeds are the Reason of this #earthquake M...,"['our, deeds, reason, earthquake, may, alla, f...",'our deeds reason earthquake may alla forgive ...,1
1,Forest fire near La Ronge Sask. Canada,"['forest, fire, near, la, range, task, ., cana...",'forest fire near la range task . canada ',1
2,All residents asked to 'shelter in place' are ...,"[``, residents, ask, 'shelter, place, ', notif...",`` residents ask 'shelter place ' notify offic...,1
3,"13,000 people receive #wildfires evacuation or...","['13,000, people, receive, wildfires, evacuati...","'13,000 people receive wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,"['just, get, send, photo, ruby, alaska, smoke,...",'just get send photo ruby alaska smoke wildfir...,1


In [31]:
test.head()

,text,tokens,tokens1
0,Just happened a terrible car crash,"['just, happen, terrible, car, crash, ']",'just happen terrible car crash '
1,"Heard about #earthquake is different cities, s...","['heard, earthquake, different, cities, ,, sta...","'heard earthquake different cities , stay safe..."
2,"there is a forest fire at spot pond, geese are...","['there, forest, fire, spot, pond, ,, geese, f...","'there forest fire spot pond , geese flee acro..."
3,Apocalypse lighting. #Spokane #wildfires,"['apocalypse, light, ., speak, wildfires, ']",'apocalypse light . speak wildfires '
4,Typhoon Soudelor kills 28 in China and Taiwan,"['typhoon, soudelor, kill, 28, china, taiwan, ']",'typhoon soudelor kill 28 china taiwan '


In [32]:
X = train[['tokens1', 'target']]
X.head()

,tokens1,target
0,'our deeds reason earthquake may alla forgive ...,1
1,'forest fire near la range task . canada ',1
2,`` residents ask 'shelter place ' notify offic...,1
3,"'13,000 people receive wildfires evacuation or...",1
4,'just get send photo ruby alaska smoke wildfir...,1


In [33]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = X.tokens1.values
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)

# encode document
# vector = vectorizer.transform(text)

{'our': 8715, 'deeds': 3418, 'reason': 9749, 'earthquake': 3997, 'may': 7550, 'alla': 909, 'forgive': 4835, 'us': 12460, 'forest': 4830, 'fire': 4690, 'near': 8207, 'la': 6829, 'range': 9679, 'task': 11605, 'canada': 2256, 'residents': 9952, 'ask': 1255, 'shelter': 10685, 'place': 9130, 'notify': 8413, 'officer': 8532, 'evacuation': 4339, 'order': 8677, 'expect': 4419, '13': 91, '000': 1, 'people': 8969, 'receive': 9765, 'wildfires': 12928, 'california': 2220, 'just': 6574, 'get': 5123, 'send': 10564, 'photo': 9057, 'ruby': 10244, 'alaska': 865, 'smoke': 10923, 'pour': 9281, 'school': 10445, 'rockyfire': 10139, 'update': 12428, 'hwy': 5892, '20': 191, 'close': 2699, 'directions': 3660, 'due': 3931, 'lake': 6852, 'county': 3056, 'flood': 4753, 'disaster': 3676, 'heavy': 5587, 'rain': 9655, 'cause': 2388, 'flash': 4726, 'streets': 11315, 'monitor': 7928, 'colorado': 2797, 'spring': 11137, 'areas': 1171, 'top': 12020, 'hill': 5668, 'see': 10527, 'woods': 13024, 'there': 11814, 'emergency'

In [34]:
train_x = vectorizer.transform(train.tokens1.values).toarray()
train_y = train.target.values
test_x = vectorizer.transform(test.tokens1.values).toarray()
print(len(train_x),len(train_y))
print(len(test_x))

7613 7613
3263


In [35]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(train_x,train_y)

BernoulliNB()

In [36]:
y_predict = clf.predict(test_x)
y_predict

array([0, 1, 1, ..., 1, 1, 1])

In [37]:
 from sklearn.model_selection import cross_val_score

In [38]:
score = cross_val_score(clf, train_x, train_y, scoring="accuracy", cv = 3)
print(score)

[0.75295508 0.70527975 0.75088687]
